In [446]:
import numpy as np
from scipy.integrate import solve_ivp
import calcephpy as efm
import julian
import datetime
import matplotlib.pyplot as plt

from conversions import Kepler2Cartesian
from Kepler import KeplerSolver
import constants

In [447]:
def f1(t, x):
    r = np.linalg.norm(x[:3])
    r3 = r**3
    yr = constants.muE / r3
    result = np.empty(6)
    result[0] = x[3]
    result[1] = x[4]
    result[2] = x[5]
    result[3] = -x[0]*yr
    result[4] = -x[1]*yr
    result[5] = -x[2]*yr
    return result

def precession_matrix(jd):
    """
    This function calculates precession matrix for conversions between reference systems
    x_old(in ICRF(J2000)) = P @ x_new(in ICRF(jd))
    
    Input:
    jd - julian date
    
    Output:
    P - np.array 3x3 (float)
    """
    t = (jd - 2451545.0)/36525.0
    t2 = t**2
    t3 = t**3
    sec2rad = np.pi / 180 / 3600
    
    z = -(2306.2181 * t + 1.09468 * t2 + 0.018203 * t3) * sec2rad
    theta = (2004.3109 * t + 0.42665 * t2 - 0.041833 * t3) * sec2rad
    dzeta = -(2306.2181 * t + 0.30188 * t2 + 0.017998 * t3) * sec2rad
    
    M1 = np.array([[np.cos(z), -np.sin(z), 0], [np.sin(z), np.cos(z), 0], [0, 0, 1.]])
    M2 = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1., 0], [-np.sin(theta), 0, np.cos(theta)]])
    M3 = np.array([[np.cos(dzeta), -np.sin(dzeta), 0], [np.sin(dzeta), np.cos(dzeta), 0], [0, 0, 1.]])
    
    return M1 @ M2 @ M3

In [448]:
date = '01.10.2021'
date = datetime.datetime(int(date[-4:]), int(date[3:5]), int(date[:2]))
jd0 = julian.to_jd(date) # midnight!

integration_time = 150 * 60 * 60
t1 = 12 * 60 * 60 # Noon
t2 = t1 + integration_time

#to plot Lunar trajectory
stepL = 100
djd = np.arange(t1, t2 + stepL, stepL) / (24*60*60)
jd = jd0 + djd // 1
djd = djd % 1

ef = efm.CalcephBin.open("epm2017.bsp")
PV = ef.compute_unit(jd, djd, efm.NaifId.MOON, efm.NaifId.EARTH, 
                    efm.Constants.UNIT_KM + efm.Constants.UNIT_SEC  + efm.Constants.USE_NAIFID)
PV = np.array(PV) # km and seconds!

P = precession_matrix(jd0).T # must take into account, that epehemerides return PV in ECI(J2000), so I want convert it into the ECI(current_date)
rL = P @ PV[:3, :] * 1000 # meters
VL = P @ PV[3:, :] * 1000
ef.close()

In [449]:
i = 51.8 / 180 * np.pi
W = 21.5 / 180 * np.pi # to be set
w = -12 / 180 * np.pi # to be set

hp = 200e3
rp = constants.RE + hp # before impulse
Vp0 = np.sqrt(constants.muE / rp)
pump = 3130
Vp1 = Vp0 + pump # after impulse
a = 1 / (2 / rp - Vp1**2 / constants.muE)
e = 1 - rp / a
p = a * (1 - e**2)

v1, E1 = KeplerSolver(a, e, t1, epoch = t1)
v2, E2 = KeplerSolver(a, e, t2, epoch = t1)
v = np.linspace(v1, v2, 1000)
x = np.zeros((6, *v.shape))
for j in range(len(v)):
    x[:, j] = Kepler2Cartesian(i, W, w, e, p, v[j])    

In [450]:
dt_eval = stepL
t_eval = np.arange(t1, t2 + dt_eval, dt_eval)

f0 = Kepler2Cartesian(i, W, w, e, p, v1)
tolerance = 1e-10

res = solve_ivp(f1, (t1, t2), f0, t_eval = t_eval, rtol = tolerance, atol = tolerance).y

R = res[0:3]
V = res[3:]

d = np.linalg.norm(rL - R, axis = 0)
re = 66194e3

In [451]:
flag = np.any(d < re)
ind = None
print(flag)
if flag:
    ind = np.where(d < re)[0][0]
    print(f"Fly-time: {(t_eval[ind] - t1) / 3600 :0.1f} hours")

True
Fly-time: 109.2 hours


In [452]:
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot(*rL, color = "orange")
ax.plot(*rL[:, 0], 'o', color = "blue")
ax.plot(*rL[:, -1], 'o', color = "red")

ax.plot(*R[:3, :], color = "green")
ax.plot(*R[:3, 0], 'o', color = "blue")
ax.plot(*R[:3, -1], 'o', color = "red")

if ind:
    ax.plot(*rL[:, ind], 'o', color = "green")
    ax.plot(*R[:3, ind], 'o', color = "green")
pass

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [453]:
np.linalg.norm(rL[:, ind]) / 1e3

368519.32864205696